In [2]:
import wrds
import yfinance as yf
db = wrds.Connection()
import pandas as pd
from datetime import datetime

!echo $HOME
!ls -la /root/.pgpass
!chmod 0600 ~/.pgpass

db.create_pgpass_file()


WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
/Users/mukeshwaranbaskaran
ls: /root/.pgpass: No such file or directory


In [3]:
# Assuming `db` is your database connection object
schema = db.describe_table(library='crsp', table='sector')  # Adjust as necessary

# Convert schema to a pandas DataFrame
schema_df = pd.DataFrame(schema)

# Display the DataFrame
schema_df


Approximately 68996960 rows in crsp.sector.


,name,nullable,type,comment
0,tradetstmp,True,TIMESTAMP,None
1,crspre1,True,"NUMERIC(8, 2)",US REIT : Price-Only Index
2,crspen1,True,"NUMERIC(8, 2)",US Oil and Gas : Price-Only Index
3,crspmt1,True,"NUMERIC(8, 2)",US Materials : Price-Only Index
4,crspid1,True,"NUMERIC(8, 2)",US Industrials : Price-Only Index
5,crspcs1,True,"NUMERIC(8, 2)",US Consumer Services : Price-Only Index
6,crsphc1,True,"NUMERIC(8, 2)",US Health Care : Price-Only Index
7,crspcd1,True,"NUMERIC(8, 2)",None
8,crspte1,True,"NUMERIC(8, 2)",US Telecom : Price-Only Index
9,crsput1,True,"NUMERIC(8, 2)",US Utilities : Price-Only Index


In [4]:
# # Retrieve the schema for the healthcare stocks
# # Assuming there is a healthcare table or related data in CRSP
# healthcare_query = """
# SELECT *
# FROM crsp.stocknames
# WHERE siccd BETWEEN 2830 AND 2839 OR siccd BETWEEN 3840 AND 3851
# """

# # Execute the query
# healthcare_stocks = db.raw_sql(healthcare_query)

# # Display the first few rows of the dataframe
# print(healthcare_stocks.head())

In [5]:
# # Retrieve financials data for NVO, including market cap
# financials_query = """
# SELECT c.gvkey, f.datadate, f.tic, f.at, f.sale, f.revt, f.cogs, f.xsga, f.dp, f.xrd, 
#        f.ib, f.ebitda, f.ebit, f.ni, f.txp, (f.at * p.prc) AS market_cap
# FROM comp.funda AS f
# JOIN comp.company AS c ON f.gvkey = c.gvkey
# JOIN crsp.dsf AS p ON p.permno = (
#     SELECT permno
#     FROM crsp.stocknames
#     WHERE ticker = 'NVO'
#       AND namedt <= f.datadate
#       AND nameenddt > f.datadate
#     LIMIT 1
# )
# WHERE f.tic = 'NVO'
# ORDER BY f.datadate DESC
# """

# financials_data = db.raw_sql(financials_query)

# print("\nFinancials Data:")
# financials_data.head()


In [6]:
# Load the list of unique tickers
tickers_list = pd.read_csv("biotech_tickers.csv").Ticker.unique().tolist()

# Define the date range
start_date = '2020-01-01'
end_date = '2024-01-01'

# Initialize an empty DataFrame to store all price data
all_price_data = pd.DataFrame()

# Function to get market caps
def get_market_caps(tickers):
    market_caps = {}
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        market_caps[ticker] = stock.info.get('marketCap', 'N/A')
    return market_caps

# Get market caps for the tickers in your list
market_caps = get_market_caps(tickers_list)

# Loop through each ticker and fetch the price data
for ticker in tickers_list:
    # SQL query to fetch price data
    query = f"""
    SELECT date, permno, prc, ret, vol
    FROM crsp.dsf
    WHERE date BETWEEN '{start_date}' AND '{end_date}'
      AND permno IN (
        SELECT permno
        FROM crsp.stocknames
        WHERE ticker = '{ticker}'
          AND namedt <= '{end_date}'
          AND nameenddt >= '{start_date}'
      )
    ORDER BY date
    """

    # Execute the query and store results in a DataFrame
    price_data = db.raw_sql(query, date_cols=['date'])

    # Add the ticker and market cap to the price data
    price_data['ticker'] = ticker
    price_data['market_cap'] = market_caps.get(ticker, 'N/A')

    # Append the retrieved data to the all_price_data DataFrame
    all_price_data = pd.concat([all_price_data, price_data], ignore_index=True)

# Display the first few rows of the combined data
all_price_data.head()

all_price_data.to_csv(f"equities_biotech_pull_{start_date}_{end_date}.csv")


/var/folders/1w/11tc25ps6ts6s9qzm42mm3j40000gn/T/ipykernel_87283/2893040834.py:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_price_data = pd.concat([all_price_data, price_data], ignore_index=True)
/var/folders/1w/11tc25ps6ts6s9qzm42mm3j40000gn/T/ipykernel_87283/2893040834.py:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_price_data = pd.concat([all_price_data, price_data], ignore_index=True)
/var/folders/1w/11tc25ps6ts6s9qzm42mm3j40000gn/T/ipykernel_87283/2893040834.py:47: FutureW

In [16]:
query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'optionm'
"""
tables = db.raw_sql(query)
print(tables)


      table_name
0    borrate1996
1    borrate1997
2    borrate1998
3    borrate1999
4    borrate2000
..           ...
407   vsurfd2021
408   vsurfd2022
409   vsurfd2023
410   zero_curve
411       zerocd

[412 rows x 1 columns]
